In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install pyspark

     |████████████████████████████████| 204.7MB 56kB/s 
     |████████████████████████████████| 204kB 44.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=791b0de2f982edd9c0a3ded1830d20e68dc664316e36fda58c0881cf4b65f009
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark import SparkContext, SparkConf

In [ ]:
!wget http://media.sundog-soft.com/hadoop/ml-100k/u.item
!wget http://media.sundog-soft.com/hadoop/ml-100k/u.data

--2020-08-09 21:43:17--  http://media.sundog-soft.com/hadoop/ml-100k/u.item
Resolving media.sundog-soft.com (media.sundog-soft.com)... 52.216.166.99
Connecting to media.sundog-soft.com (media.sundog-soft.com)|52.216.166.99|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236344 (231K) [application/octet-stream]
Saving to: ‘u.item’

u.item              100%[===================>] 230.80K  --.-KB/s    in 0.05s   

2020-08-09 21:43:17 (4.57 MB/s) - ‘u.item’ saved [236344/236344]

--2020-08-09 21:43:18--  http://media.sundog-soft.com/hadoop/ml-100k/u.data
Resolving media.sundog-soft.com (media.sundog-soft.com)... 52.216.166.99
Connecting to media.sundog-soft.com (media.sundog-soft.com)|52.216.166.99|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2079229 (2.0M) [application/octet-stream]
Saving to: ‘u.data’

u.data              100%[===================>]   1.98M  --.-KB/s    in 0.09s   

2020-08-09 21:43:18 (21.8 MB/s) - ‘u.data’ saved [20792

In [ ]:
movie_names={}
hand=open("u.item",encoding = "ISO-8859-1")
for i in hand:
  line=i.split('|')
  movie_names[int(line[0])]=line[1]

In [ ]:
## create a spark session
spark = SparkSession.builder.appName('AvgMovieRateing').getOrCreate()

In [ ]:
## read in data as RDD
line = spark.sparkContext.textFile('u.data')

In [ ]:
type(line)

pyspark.rdd.RDD

In [ ]:
## create data extracting function for next rdd.map()
def lineparse(line):
  content=line.split()
  return Row(movieID=int(content[1]),rating=float(content[2]))

In [ ]:
## parse from raw data
ratings=line.map(lineparse)

In [ ]:
## convert to Dataframe
movie_ratings=spark.createDataFrame(ratings)

In [ ]:
##  calculate average rating
avg_ratings=movie_ratings.groupBy('movieID').avg('rating')

In [ ]:
## calculate how many rate each movie has
cnt_ratings=movie_ratings.groupBy('movieID').count()

In [ ]:
## join two calculation resulats together
avg_cnt=avg_ratings.join(cnt_ratings,on='movieID')

In [ ]:
## filter out movie which have less than 10 people rated 
avg_cnt_sub=avg_cnt.filter('count>10')

In [ ]:
#avg_cnt_sub.where('count<12').collect()

In [ ]:
avg_cnt_sub.first()

Row(movieID=26, avg(rating)=3.452054794520548, count=73)

In [ ]:
top_10_rating=avg_cnt_sub.orderBy('avg(rating)').take(10)

In [ ]:
type(top_10_rating)

list

In [ ]:
top_10_rating[0]

Row(movieID=424, avg(rating)=1.3157894736842106, count=19)

In [ ]:
## print top 10 movies' ID, name, average rating and rating count
for i in top_10_rating:
  print(i[0],movie_names[i[0]],i[1],i[2])

424 Children of the Corn: The Gathering (1996) 1.3157894736842106 19
669 Body Parts (1991) 1.6153846153846154 13
440 Amityville II: The Possession (1982) 1.6428571428571428 14
758 Lawnmower Man 2: Beyond Cyberspace (1996) 1.7142857142857142 21
1274 Robocop 3 (1993) 1.7272727272727273 11
457 Free Willy 3: The Rescue (1997) 1.7407407407407407 27
1254 Gone Fishin' (1997) 1.8181818181818181 11
976 Solo (1996) 1.8333333333333333 12
545 Vampire in Brooklyn (1995) 1.8333333333333333 12
1230 Ready to Wear (Pret-A-Porter) (1994) 1.8333333333333333 18
